Importing Libraries

In [ ]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers.core import Dense, Activation, Dropout
from keras.optimizers import Adam, Nadam
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.callbacks import ModelCheckpoint
import imblearn
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.metrics import confusion_matrix
import sklearn
import tensorflow as tf
from google.colab import files
import pickle
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import pickle

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


# **Linear Regression**

In [ ]:
with open('/content/drive/My Drive/Auto-AI-2019-20/New Training Data/cond_prob/lr0.01/train_lr_freq.pickle', 'rb') as handle:
    training_dataa = pickle.load(handle)

with open('/content/drive/My Drive/Auto-AI-2019-20/New Training Data/cond_prob/lr0.01/label_lr_freq.pickle', 'rb') as handle:
    training_labelss = pickle.load(handle)

In [ ]:
X_resampled, y_resampled = BorderlineSMOTE().fit_resample(training_dataa, training_labelss)
x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=0, test_size=0.1)
#x_train, x_test, y_train, y_test = train_test_split(training_dataa, training_labelss, random_state=0, test_size=0.1)



X_train = np.array(x_train)
X_test = np.array(x_test)


#using class weights to balance the data
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights = dict(enumerate(class_weights))

# convert list of labels to binary class matrix
y_train = np_utils.to_categorical(y_train) 
y_test = np_utils.to_categorical(y_test) 



input_dim = X_train.shape[1]
nb_classes = y_train.shape[1]

# Here's an MLP
model = Sequential()
model.add(Dense(32, input_dim=input_dim))
model.add(Activation('relu'))
model.add(Dropout(0.35))

model.add(Dense(64, input_dim=input_dim))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# model.add(Dense(128))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))



model.add(Dense(nb_classes))
model.add(Activation('softmax'))
opt = Adam(learning_rate=0.0001)

model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('linreg-cp-sig075.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
print("Training...")
history = model.fit(X_train, y_train, epochs=250, batch_size=16,
                    validation_split=0.2,
                    callbacks = [es,mc],
                    verbose=1)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Training...
Epoch 1/250
48/64 [=====================>........] - ETA: 0s - loss: 0.7172 - accuracy: 0.4727
Epoch 00001: val_loss improved from inf to 0.69838, saving model to linreg-cp-sig075.h5
64/64 [==============================] - 0s 3ms/step - loss: 0.7115 - accuracy: 0.4873 - val_loss: 0.6984 - val_accuracy: 0.5430
Epoch 2/250
47/64 [=====================>........] - ETA: 0s - loss: 0.6983 - accuracy: 0.5253
Epoch 00002: val_loss improved from 0.69838 to 0.69016, saving model to linreg-cp-sig075.h5
64/64 [==============================] - 0s 2ms/step - loss: 0.7031 - accuracy: 0.5157 - val_loss: 0.6902 - val_accuracy: 0.5273
Epoch 3/250
44/64 [===================>..........] - ETA: 0s - loss: 0.6919 - accuracy: 0.5071
Epoch 00003: val_loss improved from 0.69016 to 0.68203, saving model to linreg-cp-sig075.h5
64/64 [==============================] - 0s 2ms/step - loss: 0.6914 - accuracy: 0.5127 - val_loss: 0.6820 - val_accuracy: 0.5781
Epoch 4/250
46/64 [====================>....

In [ ]:
sqr_model = tf.python.keras.models.load_model('linreg-cp-sig075.h5')
adam_predictions = sqr_model.predict(X_test, verbose=1)

5/5 [==============================] - 0s 1ms/step


In [ ]:
# prediction and clasification report
y_true, y_pred = [],[]
#classes = le.classes_
for idx, prediction in enumerate(adam_predictions): 
    y_true.append([np.argmax(y_test[idx])])
    y_pred.append([np.argmax(prediction)])
    
print(sklearn.metrics.classification_report(y_pred, y_true))

              precision    recall  f1-score   support

           0       0.83      1.00      0.91        60
           1       1.00      0.85      0.92        82

    accuracy                           0.92       142
   macro avg       0.92      0.93      0.92       142
weighted avg       0.93      0.92      0.92       142



In [ ]:
files.download('linreg-cp-sig075.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
with open('/content/drive/My Drive/Auto-AI-2019-20/New Training Data/cond_prob/lr0.01/train_lr_sq.pickle', 'rb') as handle:
    training_dataa = pickle.load(handle)

with open('/content/drive/My Drive/Auto-AI-2019-20/New Training Data/cond_prob/lr0.01/label_lr_sq.pickle', 'rb') as handle:
    training_labelss = pickle.load(handle)

In [ ]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
X_resampled, y_resampled = BorderlineSMOTE().fit_resample(training_dataa, training_labelss)
x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=0, test_size=0.1)
#x_train, x_test, y_train, y_test = train_test_split(training_dataa, training_labelss, random_state=0, test_size=0.1)



X_train = np.array(x_train)
X_test = np.array(x_test)


#using class weights to balance the data
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights = dict(enumerate(class_weights))

# weights = {0:1,1:2#}
# convert list of labels to binary class matrix
y_train = np_utils.to_categorical(y_train) 
y_test = np_utils.to_categorical(y_test) 



input_dim = X_train.shape[1]
nb_classes = y_train.shape[1]

# Here's an MLP
model = Sequential()
model.add(Dense(64, input_dim=input_dim))
model.add(Activation('relu'))
model.add(Dropout(0.35))

model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))
opt = Adam(learning_rate=0.0001)


model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('linreg-cp-sq01.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
print("Training...")
history = model.fit(X_train, y_train, epochs=250, batch_size=16, 

                    validation_split=0.2,
                    callbacks = [es,mc],
                    verbose=1)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Training...
Epoch 1/250
36/59 [=================>............] - ETA: 0s - loss: 0.7025 - accuracy: 0.4653
Epoch 00001: val_loss improved from inf to 0.68930, saving model to linreg-cp-sq01.h5
59/59 [==============================] - 0s 4ms/step - loss: 0.6978 - accuracy: 0.4861 - val_loss: 0.6893 - val_accuracy: 0.5574
Epoch 2/250
38/59 [==================>...........] - ETA: 0s - loss: 0.6939 - accuracy: 0.5395
Epoch 00002: val_loss improved from 0.68930 to 0.68625, saving model to linreg-cp-sq01.h5
59/59 [==============================] - 0s 2ms/step - loss: 0.6907 - accuracy: 0.5448 - val_loss: 0.6862 - val_accuracy: 0.5787
Epoch 3/250
35/59 [================>.............] - ETA: 0s - loss: 0.6889 - accuracy: 0.5161
Epoch 00003: val_loss improved from 0.68625 to 0.68224, saving model to linreg-cp-sq01.h5
59/59 [==============================] - 0s 2ms/step - loss: 0.6893 - accuracy: 0.5299 - val_loss: 0.6822 - val_accuracy: 0.6085
Epoch 4/250
35/59 [================>.............]

In [ ]:
sqr_model = tf.python.keras.models.load_model('linreg-cp-sq01.h5')
adam_predictions = sqr_model.predict(X_test, verbose=1)
# prediction and clasification report
y_true, y_pred = [],[]
#classes = le.classes_
for idx, prediction in enumerate(adam_predictions): 
    y_true.append([np.argmax(y_test[idx])])
    y_pred.append([np.argmax(prediction)])
    
print(sklearn.metrics.classification_report(y_pred, y_true))

5/5 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       0.81      0.98      0.89        57
           1       0.98      0.82      0.90        74

    accuracy                           0.89       131
   macro avg       0.90      0.90      0.89       131
weighted avg       0.91      0.89      0.89       131



In [ ]:
files.download('linreg-cp-sq01.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
with open('/content/drive/My Drive/Auto-AI-2019-20/New Training Data/cond_prob/lr0.01/train_lr_log.pickle', 'rb') as handle:
    training_dataa = pickle.load(handle)

with open('/content/drive/My Drive/Auto-AI-2019-20/New Training Data/cond_prob/lr0.01/label_lr_log.pickle', 'rb') as handle:
    training_labelss = pickle.load(handle)

In [ ]:
X_resampled, y_resampled = BorderlineSMOTE().fit_resample(training_dataa, training_labelss)
x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=0, test_size=0.1)
#x_train, x_test, y_train, y_test = train_test_split(training_dataa, training_labelss, random_state=0, test_size=0.1)


X_train = np.array(x_train)
X_test = np.array(x_test)


#using class weights to balance the data
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights = dict(enumerate(class_weights))

# convert list of labels to binary class matrix
y_train = np_utils.to_categorical(y_train) 
y_test = np_utils.to_categorical(y_test) 

# weights = {0:1,1:2}

input_dim = X_train.shape[1]
nb_classes = y_train.shape[1]

# Here's an MLP
model = Sequential()
model.add(Dense(32, input_dim=input_dim))
model.add(Activation('relu'))
model.add(Dropout(0.35))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# model.add(Dense(64))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))


model.add(Dense(nb_classes))
model.add(Activation('softmax'))
opt = Adam(learning_rate=0.0001)


model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('linreg-cp-log01.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
print("Training...")
history = model.fit(X_train, y_train, epochs=250, batch_size=16,
                    validation_split=0.2,
                    callbacks = [mc,es],
                    verbose=1)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Training...
Epoch 1/250
38/62 [=================>............] - ETA: 0s - loss: 0.6924 - accuracy: 0.4819
Epoch 00001: val_loss improved from inf to 0.68797, saving model to linreg-cp-log01.h5
62/62 [==============================] - 0s 4ms/step - loss: 0.6924 - accuracy: 0.5061 - val_loss: 0.6880 - val_accuracy: 0.5592
Epoch 2/250
36/62 [================>.............] - ETA: 0s - loss: 0.6932 - accuracy: 0.5278
Epoch 00002: val_loss improved from 0.68797 to 0.68351, saving model to linreg-cp-log01.h5
62/62 [==============================] - 0s 2ms/step - loss: 0.6904 - accuracy: 0.5408 - val_loss: 0.6835 - val_accuracy: 0.6082
Epoch 3/250
29/62 [=============>................] - ETA: 0s - loss: 0.6859 - accuracy: 0.5603
Epoch 00003: val_loss improved from 0.68351 to 0.67873, saving model to linreg-cp-log01.h5
62/62 [==============================] - 0s 2ms/step - loss: 0.6827 - accuracy: 0.5622 - val_loss: 0.6787 - val_accuracy: 0.6000
Epoch 4/250
36/62 [================>...........

In [ ]:
sqr_model = tf.python.keras.models.load_model('linreg-cp-log01.h5')
adam_predictions = sqr_model.predict(X_test, verbose=1)
# prediction and clasification report
y_true, y_pred = [],[]
#classes = le.classes_
for idx, prediction in enumerate(adam_predictions): 
    y_true.append([np.argmax(y_test[idx])])
    y_pred.append([np.argmax(prediction)])
    
print(sklearn.metrics.classification_report(y_pred, y_true))

5/5 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       0.88      0.92      0.90        65
           1       0.93      0.89      0.91        72

    accuracy                           0.91       137
   macro avg       0.90      0.91      0.91       137
weighted avg       0.91      0.91      0.91       137



In [ ]:
files.download('linreg-cp-log01.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
with open('/content/drive/My Drive/Auto-AI-2019-20/New Training Data/cond_prob/lr0.01/train_lr_freq.pickle', 'rb') as handle:
    training_dataa = pickle.load(handle)

with open('/content/drive/My Drive/Auto-AI-2019-20/New Training Data/cond_prob/lr0.01/label_lr_freq.pickle', 'rb') as handle:
    training_labelss = pickle.load(handle)

In [ ]:
X_resampled, y_resampled = BorderlineSMOTE().fit_resample(training_dataa, training_labelss)
x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=0, test_size=0.1)



X_train = np.array(x_train)
X_test = np.array(x_test)


#using class weights to balance the data
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights = dict(enumerate(class_weights))

#weights = {0:1,1:2}
# convert list of labels to binary class matrix
y_train = np_utils.to_categorical(y_train) 
y_test = np_utils.to_categorical(y_test) 



input_dim = X_train.shape[1]
nb_classes = y_train.shape[1]

# Here's an MLP
model = Sequential()
model.add(Dense(16, input_dim=input_dim))
model.add(Activation('relu'))
model.add(Dropout(0.35))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))


model.add(Dense(nb_classes))
model.add(Activation('softmax'))
opt = Adam(learning_rate=0.0001)


model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('linreg-cp-freq01.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
print("Training...")
history = model.fit(X_train, y_train, epochs=250, batch_size=16,
                    validation_split=0.2,
                    callbacks = [mc,es],
                    verbose=1)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Training...
Epoch 1/250
38/62 [=================>............] - ETA: 0s - loss: 0.6962 - accuracy: 0.4967
Epoch 00001: val_loss improved from inf to 0.69429, saving model to linreg-cp-freq01.h5
62/62 [==============================] - 0s 4ms/step - loss: 0.6959 - accuracy: 0.5025 - val_loss: 0.6943 - val_accuracy: 0.4715
Epoch 2/250
37/62 [================>.............] - ETA: 0s - loss: 0.6917 - accuracy: 0.5253
Epoch 00002: val_loss improved from 0.69429 to 0.69067, saving model to linreg-cp-freq01.h5
62/62 [==============================] - 0s 2ms/step - loss: 0.6917 - accuracy: 0.5198 - val_loss: 0.6907 - val_accuracy: 0.5407
Epoch 3/250
41/62 [==================>...........] - ETA: 0s - loss: 0.6930 - accuracy: 0.4817
Epoch 00003: val_loss improved from 0.69067 to 0.68757, saving model to linreg-cp-freq01.h5
62/62 [==============================] - 0s 2ms/step - loss: 0.6924 - accuracy: 0.4964 - val_loss: 0.6876 - val_accuracy: 0.5894
Epoch 4/250
39/62 [=================>.......

In [ ]:
sqr_model = tf.python.keras.models.load_model('linreg-cp-freq01.h5')
adam_predictions = sqr_model.predict(X_test, verbose=1)
# prediction and clasification report
y_true, y_pred = [],[]
#classes = le.classes_
for idx, prediction in enumerate(adam_predictions): 
    y_true.append([np.argmax(y_test[idx])])
    y_pred.append([np.argmax(prediction)])
    
print(sklearn.metrics.classification_report(y_pred, y_true))

5/5 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       0.93      0.96      0.95        72
           1       0.95      0.92      0.94        65

    accuracy                           0.94       137
   macro avg       0.94      0.94      0.94       137
weighted avg       0.94      0.94      0.94       137



In [ ]:
files.download('linreg-cp-freq01.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
with open('/content/drive/My Drive/Auto-AI-2019-20/New Training Data/cond_prob/lr75/train_lr_sqr75.pickle', 'rb') as handle:
    training_dataa = pickle.load(handle)

with open('/content/drive/My Drive/Auto-AI-2019-20/New Training Data/cond_prob/lr75/label_lr_sqr75.pickle', 'rb') as handle:
    training_labelss = pickle.load(handle)

In [ ]:
X_resampled, y_resampled = BorderlineSMOTE().fit_resample(training_dataa, training_labelss)
x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=0, test_size=0.1)



X_train = np.array(x_train)
X_test = np.array(x_test)


#using class weights to balance the data
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights = dict(enumerate(class_weights))

# convert list of labels to binary class matrix
y_train = np_utils.to_categorical(y_train) 
y_test = np_utils.to_categorical(y_test) 



input_dim = X_train.shape[1]
nb_classes = y_train.shape[1]

# Here's an MLP
model = Sequential()
model.add(Dense(32, input_dim=input_dim))
model.add(Activation('relu'))
model.add(Dropout(0.35))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))


model.add(Dense(nb_classes))
model.add(Activation('softmax'))
opt = Adam(learning_rate=0.0001)

model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('linreg-cp-sqr75.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
print("Training...")
history = model.fit(X_train, y_train, epochs=250, batch_size=16,
                    validation_split=0.2,
                    callbacks = [mc,es],
                    verbose=1)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Training...
Epoch 1/250
38/57 [===================>..........] - ETA: 0s - loss: 0.7134 - accuracy: 0.4918
Epoch 00001: val_loss improved from inf to 0.69343, saving model to linreg-cp-sqr75.h5
57/57 [==============================] - 0s 4ms/step - loss: 0.7072 - accuracy: 0.5000 - val_loss: 0.6934 - val_accuracy: 0.5000
Epoch 2/250
42/57 [=====================>........] - ETA: 0s - loss: 0.7038 - accuracy: 0.5134
Epoch 00002: val_loss improved from 0.69343 to 0.69162, saving model to linreg-cp-sqr75.h5
57/57 [==============================] - 0s 2ms/step - loss: 0.6982 - accuracy: 0.5244 - val_loss: 0.6916 - val_accuracy: 0.5619
Epoch 3/250
43/57 [=====================>........] - ETA: 0s - loss: 0.6941 - accuracy: 0.5058
Epoch 00003: val_loss improved from 0.69162 to 0.69078, saving model to linreg-cp-sqr75.h5
57/57 [==============================] - 0s 2ms/step - loss: 0.6927 - accuracy: 0.5111 - val_loss: 0.6908 - val_accuracy: 0.5619
Epoch 4/250
33/57 [================>...........

In [ ]:
sqr_model = tf.python.keras.models.load_model('linreg-cp-sqr75.h5')
adam_predictions = sqr_model.predict(X_test, verbose=1)
# prediction and clasification report
y_true, y_pred = [],[]
#classes = le.classes_

for idx, prediction in enumerate(adam_predictions): 
    y_true.append([np.argmax(y_test[idx])])
    y_pred.append([np.argmax(prediction)])
    
print(sklearn.metrics.classification_report(y_pred, y_true))

4/4 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.83      0.88      0.85        60
           1       0.89      0.83      0.86        66

    accuracy                           0.86       126
   macro avg       0.86      0.86      0.86       126
weighted avg       0.86      0.86      0.86       126



In [ ]:
files.download('linreg-cp-sqr75.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>